In [49]:
import lidar
import rosbag
from pathlib import Path
from dask.distributed import Client
from dask import delayed
import dask.dataframe as dd
import dask

client = Client()  # start distributed scheduler locally.  Launch dashboard
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

/opt/conda/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41925 instead
  http_address["port"], self.http_server.port
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
bagfile = "/workspaces/lidar/tests/testdata/big.bag"

In [2]:
test = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 100, end_frame_number = 110)

NameError: name 'bagfile' is not defined

In [15]:
start = 0
stop = 100

chunksize = 10
chunk = 0

for i in range(start,stop):
    print(i)
    if i % chunksize == 0:
        chunk = chunk + 1
        print(f"new chunk {chunk}")


0
new chunk 1
1
2
3
4
5
6
7
8
9
10
new chunk 2
11
12
13
14
15
16
17
18
19
20
new chunk 3
21
22
23
24
25
26
27
28
29
30
new chunk 4
31
32
33
34
35
36
37
38
39
40
new chunk 5
41
42
43
44
45
46
47
48
49
50
new chunk 6
51
52
53
54
55
56
57
58
59
60
new chunk 7
61
62
63
64
65
66
67
68
69
70
new chunk 8
71
72
73
74
75
76
77
78
79
80
new chunk 9
81
82
83
84
85
86
87
88
89
90
new chunk 10
91
92
93
94
95
96
97
98
99


In [8]:
%%time
test = delayed(lidar.Dataset.from_file)(Path(bagfile), start_frame_number = 0, end_frame_number = 100)

CPU times: user 1.28 ms, sys: 1.09 ms, total: 2.37 ms
Wall time: 1.79 ms


In [9]:
%%time
testx = test.compute()

CPU times: user 1.5 s, sys: 1.03 s, total: 2.53 s
Wall time: 34.7 s


In [11]:
testx.to_file(Path("/workspaces/test3"))

Files written to: /workspaces/test3/big


In [5]:
test

Delayed('from_file-a4fbf140-4a9e-45f5-8035-7873bfafb24e')

In [14]:
testx.data.x.max().compute()

AttributeError: 'list' object has no attribute 'x'

In [19]:
data2 = dd.from_delayed(testx.data)

In [21]:
data2.x.max().compute()

1.252312421798706

In [3]:

test0 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 100)


100%|██████████| 100/100 [00:47<00:00,  2.09it/s]


In [4]:
test1 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 101, end_frame_number = 200)


100%|██████████| 99/99 [00:38<00:00,  2.60it/s]


In [5]:
str(test0.start_time)

'2020-12-03 10:37:34.313459'

In [6]:
str(test1.start_time)

'2020-12-03 10:37:44.733687'

In [7]:
newdat = test0.data + test1.data
newtimes = test0.timestamps + test1.timestamps

In [14]:
long=lidar.Dataset(data=newdat,timestamps=newtimes)

In [15]:
long[150]

Frame(               x         y         z  intensity         t  reflectivity  ring  \
0       0.804142 -0.043635  0.238854       57.0         0             4     0   
1       0.810508 -0.013412  0.231400      117.0         0             8     1   
2       0.826164  0.017104  0.227178      113.0         0             8     2   
3       0.808924 -0.013357  0.198374       99.0         0             7     5   
4       0.802965  0.016287  0.188698      147.0         0            10     6   
...          ...       ...       ...        ...       ...           ...   ...   
111918  0.802611  0.045460 -0.145731       76.0  99967600             5    51   
111919  0.805425 -0.043116 -0.153207       54.0  99967600             4    52   
111920  0.783434 -0.013295 -0.156383       95.0  99967600             6    53   
111921  0.808345  0.045884 -0.178086       99.0  99967600             7    55   
111922  0.779879  0.044118 -0.202711       60.0  99967600             4    59   

        noise  range 

In [52]:
long.to_file(Path("/workspaces/test3"))

Files written to: /workspaces/test3/accc6956-af0c-4417-a95f-306bf06586ca


In [2]:
test_in = lidar.Dataset.from_file(Path("/workspaces/test3/accc6956-af0c-4417-a95f-306bf06586ca"))

In [4]:
res = test_in[0].limit("x",-0.1,0)

In [26]:
def pipeline1(dataset,i):
    return dataset[i]

In [5]:
%%time
res = []
for i in range(0,10):
    print(i)
    item = delayed(pipeline1)(test_in,i)
    res.append(item)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker

KilledWorker: ("('read-parquet-8553ab9f86d017a516147b1f51b8d92f', 117)", <Worker 'tcp://127.0.0.1:33311', name: 1, memory: 0, processing: 1>)

In [39]:
len(newtest)

199

In [7]:
test = test_in.data[5].limit("x",-0.1,0)

In [9]:
test.compute()

AttributeError: 'DataFrame' object has no attribute 'limit'

In [10]:
test = test_in.data[5].x.max()

In [12]:
test.compute()

0.8703298

ok, so it is a dataframe. Can I acutally save the frames?

In [23]:
%%time
test0 = lidar.Frame(test_in.data[5].compute()).limit("x",-0.1,0)

CPU times: user 47.4 ms, sys: 7.6 ms, total: 55 ms
Wall time: 52.5 ms


In [27]:
%%time
test0 = delayed(lidar.Frame(test_in.data[5].compute()).limit("x",-0.1,0))

CPU times: user 42.7 ms, sys: 17.7 ms, total: 60.4 ms
Wall time: 58.4 ms


In [40]:
def pipeline2(element_in):
    return lidar.Frame(element_in).limit("x",-0.1,0)

In [41]:
%%time
test0 = delayed(pipeline2)(test_in.data[5])

CPU times: user 0 ns, sys: 497 µs, total: 497 µs
Wall time: 302 µs


In [42]:
test0

Delayed('pipeline2-fde8f300-fea7-4dfd-8ddc-7663b770e3c8')

In [45]:
test0.compute()

Frame(             x         y         z  intensity         t  reflectivity  ring  \
0    -0.000529 -1.999936 -0.537865      113.0  24080120            48    60   
1    -0.000627 -2.001783 -0.230411      163.0  24129060            66    44   
2    -0.001596 -1.991060 -0.303110      137.0  24129060            55    48   
3    -0.002624 -1.979603 -0.376019      180.0  24129060            72    52   
4    -0.004649 -1.988643 -0.454840      180.0  24129060            74    56   
...        ...       ...       ...        ...       ...           ...   ...   
1795 -0.002551  1.880352 -0.340871       29.0  75819470            10    51   
1796 -0.002700  1.826340 -0.401714       32.0  75819470            11    55   
1797 -0.002361  1.835208 -0.476258      120.0  75819470            43    59   
1798 -0.002862  1.768641 -0.532268       61.0  75819470            20    63   
1799 -0.000136  1.981558  0.222407       13.0  75868160             5    19   

      noise  range  original_id  
0        31

In [47]:
res = []
for i in range(0,10):
    item = delayed(pipeline2)(test_in.data[i])
    res.append(item)

In [51]:
res

[Delayed('pipeline2-5a804d0d-603c-4450-9d74-b49e9d9bd0d5'),
 Delayed('pipeline2-8dcaef02-aa90-4bca-a420-f51a0ccf1484'),
 Delayed('pipeline2-6f7a85f0-799a-44da-b129-996c500003e7'),
 Delayed('pipeline2-a8c9355d-aa93-4cae-8806-2d0fe9127246'),
 Delayed('pipeline2-5a1b0844-90fa-4332-b2b0-d4b65948e4a0'),
 Delayed('pipeline2-13ab93dd-9a3c-4d92-985a-a02bb3090a8c'),
 Delayed('pipeline2-e86320a0-caff-4acc-a661-3da45684aadb'),
 Delayed('pipeline2-a7f9aa55-4f51-4c60-8118-e331f4e9b0d5'),
 Delayed('pipeline2-8626772a-81b0-4a68-9ee1-cc2b49607d26'),
 Delayed('pipeline2-78f781c8-fcbe-4637-b00a-72e2a184280e')]

In [52]:
reslist= dask.compute(*res)

Ok, noice, Now can I make it a new dataset

In [56]:
newdataset = lidar.Dataset(data=res, timestamps=test_in.timestamps, meta=test_in.meta )

In [64]:
newdataset.data[0].compute()

Frame(             x         y         z  intensity         t  reflectivity  ring  \
0    -0.000524 -1.980622 -0.532670      113.0  24076640            47    60   
1    -0.000626 -1.998803 -0.230067      169.0  24125640            68    44   
2    -0.001590 -1.983151 -0.301906      143.0  24125640            57    48   
3    -0.002651 -2.000234 -0.379937      188.0  24125640            77    52   
4    -0.004690 -2.006189 -0.458854      185.0  24125640            78    56   
...        ...       ...       ...        ...       ...           ...   ...   
1788 -0.002364  1.862870 -0.267647       23.0  75815010             8    47   
1789 -0.002519  1.856737 -0.336590       35.0  75815010            12    51   
1790 -0.002702  1.827317 -0.401929       35.0  75815010            12    55   
1791 -0.002362  1.836176 -0.476509      126.0  75815010            45    59   
1792 -0.002936  1.814605 -0.546100       54.0  75815010            19    63   

      noise  range  original_id  
0        13

In [61]:
newdataset[0]

TypeError: Data argument must be a DataFrame

In [60]:
res[0].compute()

Frame(             x         y         z  intensity         t  reflectivity  ring  \
0    -0.000524 -1.980622 -0.532670      113.0  24076640            47    60   
1    -0.000626 -1.998803 -0.230067      169.0  24125640            68    44   
2    -0.001590 -1.983151 -0.301906      143.0  24125640            57    48   
3    -0.002651 -2.000234 -0.379937      188.0  24125640            77    52   
4    -0.004690 -2.006189 -0.458854      185.0  24125640            78    56   
...        ...       ...       ...        ...       ...           ...   ...   
1788 -0.002364  1.862870 -0.267647       23.0  75815010             8    47   
1789 -0.002519  1.856737 -0.336590       35.0  75815010            12    51   
1790 -0.002702  1.827317 -0.401929       35.0  75815010            12    55   
1791 -0.002362  1.836176 -0.476509      126.0  75815010            45    59   
1792 -0.002936  1.814605 -0.546100       54.0  75815010            19    63   

      noise  range  original_id  
0        13

Ok, so it must be a dataframe in the res, then is shoudl be ok

In [65]:
def pipeline3(element_in):
    return lidar.Frame(element_in).limit("x",-0.1,0).data

In [77]:
%%time
res3 = []
for i in range(0,199):
    item = delayed(pipeline3)(test_in.data[i])
    res3.append(item)

CPU times: user 30.4 ms, sys: 11.6 ms, total: 42.1 ms
Wall time: 30 ms


In [78]:
newdataset3 = lidar.Dataset(data=res3, timestamps=test_in.timestamps, meta=test_in.meta )

In [79]:
newdataset3[100]

Frame(             x         y         z  intensity         t  reflectivity  ring  \
0    -0.000516 -1.949720 -0.524359       98.0  24075660            39    60   
1    -0.000628 -2.004764 -0.230754      150.0  24124640            61    44   
2    -0.001608 -2.005889 -0.305368      165.0  24124640            67    48   
3    -0.002635 -1.988445 -0.377698      192.0  24124640            78    52   
4    -0.004697 -2.009114 -0.459522      175.0  24124640            74    56   
...        ...       ...       ...        ...       ...           ...   ...   
1791 -0.002372  1.868809 -0.268501       28.0  75827850            10    47   
1792 -0.002516  1.854769 -0.336233       33.0  75827850            12    51   
1793 -0.002685  1.815597 -0.399351       45.0  75827850            15    55   
1794 -0.002366  1.839080 -0.477263       96.0  75827850            34    59   
1795 -0.002946  1.820351 -0.547829       66.0  75827850            23    63   

      noise  range  original_id  
0        17

In [81]:
len(newdataset3)

199